In [1]:
!rm -rf /Users/ryan/prosodic_data/data/english_wordforms.sqlitedict
import sys; sys.path.insert(0,'..')
from prosodic.imports import *

In [2]:
x="""
Those hours, that with gentle work did frame
The lovely gaze where every eye doth dwell,
Will play the tyrants to the very same
And that unfair which fairly doth excel;
For never-resting time leads summer on
To hideous winter, and confounds him there;
Sap checked with frost, and lusty leaves quite gone,
Beauty o’er-snowed and bareness every where:
Then were not summer’s distillation left,
A liquid prisoner pent in walls of glass,
Beauty’s effect with beauty were bereft,
Nor it, nor no remembrance what it was:
But flowers distill’d, though they with winter meet,
Leese but their show; their substance still lives sweet.
"""

In [3]:
t=Text(x)

In [4]:
t.phonemes

In [5]:
import panphon
ft = panphon.FeatureTable()

In [6]:
seg=ft.word_fts('aʊɛːz')[0]

In [7]:
seg.data

{'syl': 1,
 'son': 1,
 'cons': -1,
 'cont': 1,
 'delrel': -1,
 'lat': -1,
 'nas': -1,
 'strid': 0,
 'voi': 1,
 'sg': -1,
 'cg': -1,
 'ant': 0,
 'cor': -1,
 'distr': 0,
 'lab': -1,
 'hi': -1,
 'lo': 1,
 'back': -1,
 'round': -1,
 'velaric': -1,
 'tense': 1,
 'long': -1,
 'hitone': 0,
 'hireg': 0}